### [Optional] Download collected dataset

In [1]:
'''
If you want to use the collected dataset, please download it from Hugging Face.
'''
!git clone https://huggingface.co/datasets/Jeongeun/omy_pnp_language

fatal: 目标路径 'omy_pnp_language' 已经存在，并且不是一个空目录。


# Visualize your data

<img src="./media/data_v2.gif" width="480" height="360">

Visualize your action based on the reconstructed simulation scene. 

The main simulation is replaying the action.

The overlayed images on the top right and bottom right are from the dataset. 

In [2]:
# Cell 1 - 设置环境变量（必须第一个运行）
import os

# 1. 设置DISPLAY
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 2. 强制使用GPU渲染（关键！）
os.environ['MUJOCO_GL'] = 'egl'  # EGL后端GPU加速
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# 3. NVIDIA GPU优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'  # 关闭垂直同步
os.environ['__GL_YIELD'] = 'NOTHING'      # 减少CPU等待
print("✓ NVIDIA GPU优化已启用")

# 4. OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'        # 关闭抗锯齿
os.environ['__GL_LOG_MAX_ANISO'] = '0'    # 关闭各向异性过滤
print("✓ OpenGL性能优化已启用")

✓ DISPLAY设置为: :0
✓ MUJOCO_GL: egl (GPU硬件加速)
✓ NVIDIA GPU优化已启用
✓ OpenGL性能优化已启用


In [3]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
import numpy as np
from lerobot.common.datasets.utils import write_json, serialize_dict
ROOT = "./demo_data_language" # The root directory to save the demonstrations 
# If you have downloaded the dataset from Hugging Face, you can set the root to the directory where the dataset is stored
# ROOT = './omy_pnp_language' # if you want to use the example data provided, root = './omy_pnp_language' instead!
dataset = LeRobotDataset('omy_pnp_language', root=ROOT) # if youu want to use the example data provided, root = './omy_pnp_language' instead!

# If you want to use the collected dataset, please download it from Hugging Face.
# dataset = LeRobotDataset('omy_pnp_language', root='omy_pnp_language')

/home/wzy/envs/lerobot-mujoco/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 6931 examples [00:00, 8222.62 examples/s]


## Load Dataset

In [4]:
import torch

class EpisodeSampler(torch.utils.data.Sampler):
    """
    Sampler for a single episode
    """
    def __init__(self, dataset: LeRobotDataset, episode_index: int):
        from_idx = dataset.episode_data_index["from"][episode_index].item()
        to_idx = dataset.episode_data_index["to"][episode_index].item()
        self.frame_ids = range(from_idx, to_idx)

    def __iter__(self):
        return iter(self.frame_ids)

    def __len__(self) -> int:
        return len(self.frame_ids)

In [5]:
# Select an episode index that you want to visualize
episode_index = 0

episode_sampler = EpisodeSampler(dataset, episode_index)
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=1,
    batch_size=1,
    sampler=episode_sampler,
)


## Visualize your Dataset on Simulation

In [6]:
from mujoco_env.y_env2 import SimpleEnv2
xml_path = './asset/example_scene_y2.xml'
PnPEnv = SimpleEnv2(xml_path, action_type='joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[23]
 [0/23] [world] mass:[0.00]kg
 [1/23] [front_object_table] mass:[1.00]kg
 [2/23] [camera] mass:[0.00]kg
 [3/23] [camera2] mass:[0.00]kg
 [4/23] [camera3] mass:[0.00]kg
 [5/23] [link1] mass:[2.06]kg
 [6/23] [link2] mass:[3.68]kg
 [7/23] [link3] mass:[2.39]kg
 [8/23] [link4] mass:[1.40]kg
 [9/23] [link5] mass:[1.40]kg
 [10/23] [link6] mass:[0.65]kg
 [11/23] [camera_center] mass:[0.00]kg
 [12/23] [tcp_link] mass:[0.32]kg
 [13/23] [rh_p12_rn_r1] mass:[0.07]kg
 [14/23] [rh_p12_rn_r2] mass:[0.02]kg
 [15/23] [rh_p12_rn_l1] mass:[0.07]kg
 [16/23] [rh_p12_rn_l2] mass:[0.02]kg
 [17/23] [body_obj_mug_5] mass:[0.00]kg
 [18/23] [object_mug_5] mass:[0.08]kg
 [19/23] [body_obj_plate_11] mass:[0.00]kg
 [20/23] [object_plate_11] mass:[0.10]kg
 [21/23] [body_obj_mug_6] mass:[0.00]kg
 [22/23] [object_mug

In [7]:
step = 0
iter_dataloader = iter(dataloader)
PnPEnv.reset()

while PnPEnv.env.is_viewer_alive():
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # Get the action from dataset
        data = next(iter_dataloader)
        if step == 0:
            # Reset the object pose based on the dataset
            instruction = data['task'][0]
            PnPEnv.set_instruction(instruction)
            PnPEnv.set_obj_pose(data['obj_init'][0,:3], data['obj_init'][0,3:6], data['obj_init'][0,6:9])
        # Get the action from dataset
        action = data['action'].numpy()
        obs = PnPEnv.step(action[0])

        # Visualize the image from dataset to rgb_overlay
        PnPEnv.rgb_agent = data['observation.image'][0].numpy()*255
        PnPEnv.rgb_ego = data['observation.wrist_image'][0].numpy()*255
        PnPEnv.rgb_agent = PnPEnv.rgb_agent.astype(np.uint8)
        PnPEnv.rgb_ego = PnPEnv.rgb_ego.astype(np.uint8)
        # 3 256 256 -> 256 256 3
        PnPEnv.rgb_agent = np.transpose(PnPEnv.rgb_agent, (1,2,0))
        PnPEnv.rgb_ego = np.transpose(PnPEnv.rgb_ego, (1,2,0))
        PnPEnv.rgb_side = np.zeros((480, 640, 3), dtype=np.uint8)
        PnPEnv.render()
        step += 1

        if step == len(episode_sampler):
            # start from the beginning
            iter_dataloader = iter(dataloader)
            PnPEnv.reset()
            step = 0
    # PnPEnv

DONE INITIALIZATION


DONE INITIALIZATION
DONE INITIALIZATION
DONE INITIALIZATION
DONE INITIALIZATION
DONE INITIALIZATION


KeyboardInterrupt: 

In [8]:
PnPEnv.env.close_viewer()

In [9]:
dataset.push_to_hub(upload_large_folder=True)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-69082c5e-3a3bdf7c22b95a6243c09f78;07383d9f-2cf4-43f3-8c3b-576fad327468)

Invalid username or password.